In [1]:
# Downgrade PyTorch to version before the weights_only=True change
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121

# Restart runtime after this
print("PyTorch downgraded. Please restart runtime and try training again.")

Looking in indexes: https://download.pytorch.org/whl/cu121
PyTorch downgraded. Please restart runtime and try training again.


In [2]:
!nvidia-smi

Fri Jul 11 15:22:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [4]:
# Add this cell before your training command
import torch
from ultralytics.nn.tasks import YOLOv10DetectionModel

# Add YOLOv10DetectionModel to PyTorch's safe globals
torch.serialization.add_safe_globals([YOLOv10DetectionModel])

print("YOLOv10DetectionModel added to safe globals successfully!")
print("Now you can run your training command.")

YOLOv10DetectionModel added to safe globals successfully!
Now you can run your training command.


In [5]:
!pip install -q git+https://github.com/THU-MIG/yolov10.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
!pip install -q supervision roboflow

In [7]:
import os
import glob
import random
import shutil

# --- Configuration ---
# Path to your zipped dataset
zip_file_path = '/content/obj.zip'
# Directory to unzip your data into
source_dir = '/content/Gh'

# Define the split ratios
# 70% for training, 20% for validation, 10% for testing
train_split = 0.7
valid_split = 0.2
# The rest will be for testing

# --- Step 1: Unzip the dataset ---
print(f"Unzipping {zip_file_path}...")
# Use -o to overwrite existing files without asking
os.system(f'unzip -o {zip_file_path} -d {source_dir}')
print("Unzipping complete.")

# --- Step 2: Define and create the required directory structure ---
base_path = '/content/'
dirs_to_create = {
    'train_images': os.path.join(base_path, 'train/images'),
    'train_labels': os.path.join(base_path, 'train/labels'),
    'valid_images': os.path.join(base_path, 'valid/images'),
    'valid_labels': os.path.join(base_path, 'valid/labels'),
    'test_images': os.path.join(base_path, 'test/images'),
    'test_labels': os.path.join(base_path, 'test/labels'),
}

for path in dirs_to_create.values():
    os.makedirs(path, exist_ok=True)

print("Directory structure created successfully.")

# --- Step 3: Get all image files and split them ---
all_image_files = sorted([f for f in glob.glob(os.path.join(source_dir, '*.jpg'))])
random.seed(42) # for reproducibility
random.shuffle(all_image_files)

# Calculate split indices
train_end_idx = int(len(all_image_files) * train_split)
valid_end_idx = train_end_idx + int(len(all_image_files) * valid_split)

# Create file lists for each set
train_files = all_image_files[:train_end_idx]
valid_files = all_image_files[train_end_idx:valid_end_idx]
test_files = all_image_files[valid_end_idx:]

# --- Step 4: Function to move files ---
def move_files(file_list, img_dest_path, lbl_dest_path):
    for img_file_path in file_list:
        # Construct the corresponding label file path
        base_filename = os.path.basename(img_file_path)
        lbl_filename = os.path.splitext(base_filename)[0] + '.txt'
        lbl_file_path = os.path.join(os.path.dirname(img_file_path), lbl_filename)

        # Move the image and label files
        if os.path.exists(lbl_file_path):
            shutil.move(img_file_path, os.path.join(img_dest_path, base_filename))
            shutil.move(lbl_file_path, os.path.join(lbl_dest_path, lbl_filename))
        else:
            print(f"Warning: Label file not found for {img_file_path}")

# --- Step 5: Move the files into the new structure ---
print("Moving training files...")
move_files(train_files, dirs_to_create['train_images'], dirs_to_create['train_labels'])
print("Moving validation files...")
move_files(valid_files, dirs_to_create['valid_images'], dirs_to_create['valid_labels'])
print("Moving test files...")
move_files(test_files, dirs_to_create['test_images'], dirs_to_create['test_labels'])

print("\nData organization complete!")
print(f"Total images: {len(all_image_files)}")
print(f"  - Training:   {len(train_files)} images")
print(f"  - Validation: {len(valid_files)} images")
print(f"  - Testing:    {len(test_files)} images")

# --- Step 6: Create the data.yaml file inside ScaledYOLOv4 ---
yaml_content = f"""
# Dataset paths relative to the ScaledYOLOv4 directory
train: ../train/images
val: ../valid/images
test: ../test/images

# number of classes
nc: 1 # <-- IMPORTANT: Change this to your number of classes (e.g., 1 for 'mask')

# class names
names: ['mask'] # <-- IMPORTANT: Change this to your class name(s)
"""

yaml_file_path = '/content/Gh/custom_data.yaml'
with open(yaml_file_path, 'w') as f:
    f.write(yaml_content)

print(f"\nSuccessfully created '{yaml_file_path}'")
print("--> IMPORTANT: Please open this file and edit 'nc' and 'names' to match your dataset!")

Unzipping /content/obj.zip...
Unzipping complete.
Directory structure created successfully.
Moving training files...
Moving validation files...
Moving test files...

Data organization complete!
Total images: 1510
  - Training:   1057 images
  - Validation: 302 images
  - Testing:    151 images

Successfully created '/content/Gh/custom_data.yaml'
--> IMPORTANT: Please open this file and edit 'nc' and 'names' to match your dataset!


In [8]:
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10b.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10x.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10l.pt
!ls -lh {HOME}/weights

total 1.2G
-rw-r--r-- 1 root root  80M May 26  2024 yolov10b.pt
-rw-r--r-- 1 root root  80M May 26  2024 yolov10b.pt.1
-rw-r--r-- 1 root root  80M May 26  2024 yolov10b.pt.2
-rw-r--r-- 1 root root 100M May 26  2024 yolov10l.pt
-rw-r--r-- 1 root root 100M May 26  2024 yolov10l.pt.1
-rw-r--r-- 1 root root 100M May 26  2024 yolov10l.pt.2
-rw-r--r-- 1 root root  64M May 26  2024 yolov10m.pt
-rw-r--r-- 1 root root  64M May 26  2024 yolov10m.pt.1
-rw-r--r-- 1 root root  64M May 26  2024 yolov10m.pt.2
-rw-r--r-- 1 root root  11M May 26  2024 yolov10n.pt
-rw-r--r-- 1 root root  11M May 26  2024 yolov10n.pt.1
-rw-r--r-- 1 root root  11M May 26  2024 yolov10n.pt.2
-rw-r--r-- 1 root root  32M May 26  2024 yolov10s.pt
-rw-r--r-- 1 root root  32M May 26  2024 yolov10s.pt.1
-rw-r--r-- 1 root root  32M May 26  2024 yolov10s.pt.2
-rw-r--r-- 1 root root 123M May 26  2024 yolov10x.pt
-rw-r--r-- 1 root root 123M May 26  2024 yolov10x.pt.1
-rw-r--r-- 1 root root 123M May 26  2024 yolov10x.pt.2


In [9]:
# STEP 1: Patch torch.load to fix the weights_only issue
import torch

# Store the original torch.load function
_original_torch_load = torch.load

def patched_torch_load(f, map_location=None, pickle_module=None, **kwargs):
    """Patched torch.load that sets weights_only=False by default"""
    # If weights_only is not explicitly set, set it to False
    if 'weights_only' not in kwargs:
        kwargs['weights_only'] = False
    return _original_torch_load(f, map_location=map_location, pickle_module=pickle_module, **kwargs)

# Replace torch.load with our patched version
torch.load = patched_torch_load

print("✅ torch.load has been patched successfully!")
print("Now you can run your YOLOv10 training command.")

✅ torch.load has been patched successfully!
Now you can run your YOLOv10 training command.


In [12]:
import os
import glob

# IMPORTANT: Set this to the number of classes in your data.yaml file
NUM_CLASSES = 1

def check_labels(directory):
    error_found = False
    label_files = glob.glob(os.path.join(directory, '**', '*.txt'), recursive=True)

    if not label_files:
        print(f"Warning: No .txt label files found in {directory}")
        return

    for file_path in label_files:
        try:
            with open(file_path, 'r') as f:
                for i, line in enumerate(f):
                    parts = line.strip().split()
                    if not parts:
                        continue

                    # Check class index
                    class_id = int(parts[0])
                    if class_id >= NUM_CLASSES:
                        print(f"❌ ERROR in {file_path} (line {i+1}):")
                        print(f"   Class index '{class_id}' is out of bounds. It must be less than nc ({NUM_CLASSES}).")
                        error_found = True

                    # Check coordinates
                    coords = [float(p) for p in parts[1:]]
                    if any(c < 0.0 or c > 1.0 for c in coords):
                        print(f"❌ ERROR in {file_path} (line {i+1}):")
                        print(f"   Bounding box coordinate '{coords}' is out of the [0, 1] range.")
                        error_found = True

        except Exception as e:
            print(f"❌ ERROR reading {file_path}: {e}")
            error_found = True

    return error_found

print("--- Checking Training Labels ---")
train_errors = check_labels('/content/train/labels/')
print("\n--- Checking Validation Labels ---")
valid_errors = check_labels('/content/valid/labels/')

if not train_errors and not valid_errors:
    print("\n✅ All labels seem to be formatted correctly!")

--- Checking Training Labels ---
❌ ERROR in /content/train/labels/unmasked (1471).txt (line 1):
   Class index '1' is out of bounds. It must be less than nc (1).
❌ ERROR in /content/train/labels/with_mask (4).txt (line 3):
   Class index '1' is out of bounds. It must be less than nc (1).
❌ ERROR in /content/train/labels/with_mask (4).txt (line 4):
   Class index '1' is out of bounds. It must be less than nc (1).
❌ ERROR in /content/train/labels/unmasked (1636).txt (line 1):
   Class index '1' is out of bounds. It must be less than nc (1).
❌ ERROR in /content/train/labels/unmasked (1516).txt (line 1):
   Class index '1' is out of bounds. It must be less than nc (1).
❌ ERROR in /content/train/labels/unmasked (1419).txt (line 1):
   Class index '1' is out of bounds. It must be less than nc (1).
❌ ERROR in /content/train/labels/63.txt (line 1):
   Class index '1' is out of bounds. It must be less than nc (1).
❌ ERROR in /content/train/labels/unmasked (1271).txt (line 1):
   Class index '1'

In [13]:
%%writefile /content/Gh/custom_data.yaml
# Dataset paths should be correct
train: ../train/images
val: ../valid/images
test: ../test/images

# IMPORTANT: Changed nc to 2 for two classes
nc: 2

# IMPORTANT: Added names for both classes
# Class 0 will be 'with_mask', Class 1 will be 'unmasked'
names: ['with_mask', 'unmasked']

Overwriting /content/Gh/custom_data.yaml


In [16]:
!yolo task=detect mode=train epochs=50 batch=32 plots=True \
model={HOME}/weights/yolov10n.pt \
data=/content/Gh/custom_data.yaml


/usr/local/lib/python3.11/dist-packages/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")
New https://py